In [1]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch 
from torch import nn
from torch.utils.data import Dataset, DataLoader

from scipy import ndimage as ndi
from scipy import fft, ifft
import scipy

from skimage import data
from skimage import io, transform, color
from skimage.util import img_as_float, pad
from skimage.filters import gabor_kernel

import pickle 
import os
import sys
import h5py
import warnings
import time
import json
import nibabel as nib

import gfb_utils as g
import nsd_utils as n

sys.path.append('/user_data/mmhender/coco_annot/cocoapi/PythonAPI/')
from pycocotools.coco import COCO


In [2]:
# Define directories and what sessions/subjects to load

code_dir = '/user_data/mmhender/test_code/'
nsd_dir = '/lab_data/tarrlab/common/datasets/NSD/'

trials_per_sess = 750
nsess = 40
trials_per_subj = trials_per_sess*nsess
subj = 1
sess = 3

subj_data_dir = os.path.join(nsd_dir,
                 'nsddata_betas','ppdata','subj%02d'%subj,'func1pt8mm',
                             'betas_fithrf_GLMdenoise_RR')
subj_roi_dir = os.path.join(nsd_dir, 'nsddata','ppdata','subj%02d'%subj,
                           'func1pt8mm','roi')

In [3]:
# Initalize the dataset here
subj = 1
roi_label = 1
sess=2

roi_filename='lh.prf-visualrois.nii.gz'

roi_voxels = n.get_roi_voxels(subj, roi_label, roi_filename)

In [ ]:
subj=1
batch_size=10

dset = n.nsd_dataset(subj, voxel_inds = roi_voxels)
trn, val, trn_subset = n.get_dataset_splits(dset1, subset_trn=True, rndseed = 983458)
valgenerator = n.nsd_dataloader(val1, batch_size=batch_size, shuffle=True, rndseed = 858875)

item = valgenerator.__next__()

In [27]:
trial_inds = np.tile(np.arange(0,10),[2])
which_sess = np.repeat(np.arange(1,3), 10)

which_sess

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [47]:
nsd_dir = '/lab_data/tarrlab/common/datasets/NSD/'
subj_data_dir = os.path.join(nsd_dir,
                 'nsddata_betas','ppdata','subj%02d'%subj,'func1pt8mm',
                             'betas_fithrf_GLMdenoise_RR')
se=1

trial_inds = np.tile(np.arange(0,10),[2])
which_sess = np.repeat(np.arange(1,3), 10)
trials_per_sess=10

voxel_inds = roi_voxels
big_array = np.zeros([20,np.shape(voxel_inds)[1]])

for se in np.arange(1,3):
    
    trials_this_sess = (trial_inds[which_sess==se]).tolist()
    print(type(trials_this_sess))
    print(trials_this_sess)
    
    t = time.time()

    hd5_fn = os.path.join(subj_data_dir,'betas_session%02d.hdf5'%se)

    data_this_sess = h5py.File(hd5_fn, "r").get('betas')[0:10,:,:,:]
    print(type(data_this_sess))
    elapsed = time.time() - t
    print('h5py loading time is %.2f'%elapsed)
    np.shape(data_this_sess)
  
    
    t = time.time()
#     data_this_sess = data_this_sess[:,voxel_inds[0], voxel_inds[1], voxel_inds[2]]
    data_this_sess = data_this_sess
    elapsed = time.time() - t
    print('first indexing time  is %.2f'%elapsed)
    print(type(data_this_sess))
        
    t = time.time()
#     big_array[which_sess==se,:] = data_this_sess[trials_this_sess,:,:,:]
    big_array[which_sess==se,:] = data_this_sess[:,voxel_inds[0], voxel_inds[1], voxel_inds[2]]
    elapsed = time.time() - t
    print('second indexing time  is %.2f'%elapsed)
    

<class 'list'>
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
<class 'numpy.ndarray'>
h5py loading time is 5.19
first indexing time  is 0.00
<class 'numpy.ndarray'>
second indexing time  is 0.00
<class 'list'>
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
<class 'numpy.ndarray'>
h5py loading time is 5.11
first indexing time  is 0.00
<class 'numpy.ndarray'>
second indexing time  is 0.00


In [65]:
t = time.time()
beta_data_set = h5py.File(hd5_fn, 'r')
values = np.copy(beta_data_set['betas'])
elapsed = time.time() - t
print('h5py loading time is %.2f'%elapsed)

print (values.dtype, np.min(values), np.max(values), values.shape)

h5py loading time is 9.76
int16 -32768 32767 (750, 83, 104, 81)


In [66]:
t = time.time()
beta_data_set = h5py.File(hd5_fn, 'r')
values = np.array(beta_data_set['betas'])
elapsed = time.time() - t
print('h5py loading time is %.2f'%elapsed)

print (values.dtype, np.min(values), np.max(values), values.shape)

h5py loading time is 9.36
int16 -32768 32767 (750, 83, 104, 81)


In [63]:
nsd_dir = '/lab_data/tarrlab/common/datasets/NSD/'
subj_data_dir = os.path.join(nsd_dir,
                 'nsddata_betas','ppdata','subj%02d'%subj,'func1pt8mm',
                             'betas_fithrf_GLMdenoise_RR')
se=2
nii_fn = os.path.join(subj_data_dir,'betas_session%02d.nii.gz'%se)
nii_volume = nib.load(nii_fn)
np.shape(nii_volume)

# nii_volume = nii_volume[:,:,:,0]
nii_volume_reshaped = np.moveaxis(nii_volume, 3,0)
# np.shape(nii_volume_reshaped)

AxisError: source: axis 3 is out of bounds for array of dimension 0

In [56]:
nii

()

In [31]:
nsd_dir = '/lab_data/tarrlab/common/datasets/NSD/'
subj_data_dir = os.path.join(nsd_dir,
                 'nsddata_betas','ppdata','subj%02d'%subj,'func1pt8mm',
                             'betas_fithrf_GLMdenoise_RR')
se=1

trial_inds = np.tile(np.arange(0,10),[2])
which_sess = np.repeat(np.arange(1,3), 10)
trials_per_sess=10

voxel_inds = roi_voxels
big_array = np.zeros([20,np.shape(voxel_inds)[1]])

for se in np.arange(1,3):
    
    trials_this_sess = trial_inds[which_sess==se]
#     print(type(trials_this_sess))
#     print(trials_this_sess)
    
    t = time.time()

    nii_fn = os.path.join(subj_data_dir,'betas_session%02d.nii.gz'%se)

    t = time.time()
    data_this_sess = nib.load(nii_fn)
    elapsed = time.time() - t
    print('nii loading time is %.2f'%elapsed)

    print(np.type(data_this_sess)
    np.shape(data_this_sess)
  
    
    t = time.time()
#     data_this_sess = data_this_sess[:,voxel_inds[0], voxel_inds[1], voxel_inds[2]]
    data_this_sess = data_this_sess[:,:,:,trials_this_sess]
    elapsed = time.time() - t
    print('first indexing time  is %.2f'%elapsed)
    print(type(data_this_sess))
        
    t = time.time()
#     big_array[which_sess==se,:] = data_this_sess[trials_this_sess,:,:,:]
    big_array[which_sess==se,:] = data_this_sess[voxel_inds[0], voxel_inds[1], voxel_inds[2],:]
    elapsed = time.time() - t
    print('second indexing time  is %.2f'%elapsed)

(10, 83, 104, 81)

In [57]:
nsd_dir = '/lab_data/tarrlab/common/datasets/NSD/'
subj_data_dir = os.path.join(nsd_dir,
                 'nsddata_betas','ppdata','subj%02d'%subj,'func1pt8mm',
                             'betas_fithrf_GLMdenoise_RR')
se=2
nii_fn = os.path.join(subj_data_dir,'betas_session%02d.nii.gz'%se)

t = time.time()
data_this_sess = nib.load(nii_fn).get_fdata()
elapsed = time.time() - t
print('nii loading time is %.2f'%elapsed)
np.shape(data_this_sess)

nii loading time is 9.79


(81, 104, 83, 750)

In [4]:
class nsd_dataset(Dataset):
  
  """
  Custom dataset class that loads NSD data (voxels), and deals with corresponding image labels.
  Returns a Pytorch dataset object. Can access items with <object>[indices].
  Items are objects with fields 'data' [nTrials x nVoxels]
  and 'subject_df' which is a pandas dataframe listing everything about those trials. 
  """ 

  def __init__(self, subj, voxel_inds, trials_per_sess=750, num_sess = 40,  
               nsd_dir = '/lab_data/tarrlab/common/datasets/NSD/',
               nsd_meta_file = '/lab_data/tarrlab/common/datasets/NSD/nsddata/experiments/nsd/nsd_stim_info_merged.pkl'):

    self.subj = subj;
    self.voxel_inds = voxel_inds;
    self.num_vox = np.shape(voxel_inds)[1]
   
    self.trials_per_sess=trials_per_sess
    self.num_sess=num_sess
    
    subj_data_dir = os.path.join(nsd_dir,
                 'nsddata_betas','ppdata','subj%02d'%subj,'func1pt8mm',
                             'betas_fithrf_GLMdenoise_RR')
    
    self.subj_data_dir = subj_data_dir
    self.nsd_meta_file = nsd_meta_file

  def __len__(self):

    if hasattr(self, 'data'):
        mylen = np.shape(self.data)[0]
    else:
        mylen = self.num_sess*self.trials_per_sess
        
    return mylen

  def __getitem__(self, trial_inds):

    if hasattr(trial_inds, '__len__')==True:
        nTrials = len(trial_inds)
    else:
        nTrials=1

    trial_inds = np.squeeze(np.array([trial_inds]))
    
    which_sess = np.floor(trial_inds/self.trials_per_sess)+1
    sess2load = np.unique(which_sess)
    data  = np.zeros([nTrials, self.num_vox])
    
    # looping over all the sessions we need data from
    for se in sess2load:
        
        trials_this_sess = (trial_inds[which_sess==se] - (se-1)*self.trials_per_sess).astype('int')        
#         print(trials_this_sess)
        
        # Load in the data for this session (slow)
        hd5_fn = os.path.join(self.subj_data_dir,'betas_session%02d.hdf5'%se)
#         print('loading from %s'%nii_fn)
        
        if len(trials_this_sess)==1:
#             print('loading trial of interest')
            # if just one trial in this session, can make this a bit faster by indexing rather than loading all data.
            # otherwise will load whole file then choose trials of interest.
            t = time.time()
            data_this_sess = h5read
#             with h5py.File(nii_fn, "r") as f:
#                 data_this_sess = np.array(f['betas'][trials_this_sess,:,:,:])
            elapsed = time.time() - t
            print('h5py loading time is %.2f'%elapsed)
        else:
#             print('loading whole file for this sess')
            t = time.time()
            with h5py.File(nii_fn, "r") as f:
                data_this_sess = np.array(f['betas'])
            elapsed = time.time() - t
#             print('h5py loading time is %.2f'%elapsed)

            # grab just the trials of interest
            data_this_sess = data_this_sess[trials_this_sess,:,:,:]

        # adjusting for the 300x scaling
        data_this_sess = data_this_sess/300
        
        # All NIFTI files in the prepared NSD data are in LPI ordering
        # (the first voxel is Left, Posterior, and Inferior)
        data_this_sess = data_this_sess[:,self.voxel_inds[0],self.voxel_inds[1],self.voxel_inds[2]]
        
#         print(which_sess)
        # putting data from this session into the correct rows of our big matrix
        data[which_sess==se, :] = data_this_sess
        
    assert(np.shape(data)[0]==nTrials)

    df = get_subj_stim_seq(self.subj).loc[trial_inds.astype(int)]
                            
    item = {'data': data, 'subject_df': df}

    return item 